In [112]:
import pandas as pd

In [ ]:
#Extration des données relatives au GDP (Données Eurostat)

'''
Informations générales
Online data code:namq_10_gdp
Unit of measure: Current prices-million euro
'''

df_GDP= pd.read_csv("Données_extraites/GDP_trimestriel_eurostat.csv", encoding='utf-8')
#print(df_GDP.head())


# Choix des colonnes rélévantes
df_GDP_selected = df_GDP[['geo', 'TIME_PERIOD', 'OBS_VALUE']]
#print(df_GDP_selected.head())

# Obtention des données de panel
df_GDP_pivot = df_GDP_selected.pivot_table(index='geo', columns='TIME_PERIOD', values='OBS_VALUE', aggfunc='first')

# Renomer l'index des colonnes
df_GDP_pivot = df_GDP_pivot.rename_axis(index={'geo': 'COUNTRY'})

# Vérification
#print(df_GDP_pivot.head())
df_GDP_pivot





Dans ce qui suit: nous allons essayer de concatener trois datasets:
- 2 comportant des données trimestrielles (1998-2008 et 2008-2024)
- 1 comporatant des données annuelles (1995-1998)

Nous avons gardé la même source (Eurostat) afin d'avoir des estimations homogènes


In [ ]:
#Extration des données Working Hours per year (Données Eurostat: 1995-1998)
'''
Informations générales
Lien:https://doi.org/10.2908/LFSA_EWHANA
Code: lfsa_ewhana
'''

df_working_hours_eurostat_an= pd.read_csv("Données_extraites/Working_hours_eurostat_1995-1998.csv", encoding='utf-8')
#print(df_GDP.head())

# Choix des colonnes rélévantes
df_working_hours_selected_eurostat_an= df_working_hours_eurostat_an[['geo', 'TIME_PERIOD', 'OBS_VALUE']]
#print(df_GDP_selected.head())

# Obtention des données de panel
df_working_hours_pivot_eurostat_an = df_working_hours_selected_eurostat_an.pivot_table(index='geo', columns='TIME_PERIOD', values='OBS_VALUE', aggfunc='first')

# Renomer l'index des colonnes
df_working_hours_pivot_eurostat_an= df_working_hours_pivot_eurostat_an.rename_axis(index={'geo': 'COUNTRY'})

# Vérification
#print(df_GDP_pivot.head())
df_working_hours_pivot_eurostat_an




In [ ]:
#Interpolation (Données annuelles--> trimestrielles)

'''
à corriger 
'''
df_working_hours_pivot_eurostat_an.columns=pd.to_datetime(
    df_working_hours_pivot_eurostat_an.columns.astype(str), format='%Y'
)

df_working_hours_pivot_eurostat_an=df_working_hours_pivot_eurostat_an.T

df_working_hours_pivot_eurostat_an= df_working_hours_pivot_eurostat_an.asfreq('Q')

df_working_hours_pivot_eurostat_an_interpolation=df_working_hours_pivot_eurostat_an.interpolate(method='linear')

df_working_hours_pivot_eurostat_an_interpolation.index = (
    df_working_hours_pivot_eurostat_an_interpolation.index.to_period('Q'))

df_working_hours_pivot_eurostat_an_interpolation=df_working_hours_pivot_eurostat_an_interpolation.T

print(df_working_hours_pivot_eurostat_an_interpolation)




In [ ]:
#Extration des données Working Hours per year (Données Eurostat à partir de 2008)

'''
Informations générales: données trimestrielles à partir de 2008
Online data code:lfsq_ewhan2
Unit of measure_ Hours per year per person
'''

df_working_hours_eurostat_1= pd.read_csv("Données_extraites/Working_hours_eurostat.csv", encoding='utf-8')
#print(df_GDP.head())

# Choix des colonnes rélévantes
df_working_hours_selected_eurostat_1= df_working_hours_eurostat_1[['geo', 'TIME_PERIOD', 'OBS_VALUE']]
#print(df_GDP_selected.head())

# Obtention des données de panel
df_working_hours_pivot_eurostat_1 = df_working_hours_selected_eurostat_1.pivot_table(index='geo', columns='TIME_PERIOD', values='OBS_VALUE', aggfunc='first')

# Renomer l'index des colonnes
df_working_hours_pivot_eurostat_1= df_working_hours_pivot_eurostat_1.rename_axis(index={'geo': 'COUNTRY'})

# Vérification
#print(df_GDP_pivot.head())
df_working_hours_pivot_eurostat_1

In [ ]:
#Extration des données Working Hours per year (Données Eurostat à partir de 1998-2008)

'''
Informations générales: données trimestrielles à partir de 2008
Online data code:lfsq_ewhana
source: https://ec.europa.eu/eurostat/databrowser/view/lfsq_ewhana__custom_15382513/default/table?lang=en
Unit of measure_ Hours per year per person
'''

df_working_hours_eurostat_2= pd.read_csv("Données_extraites/Working_hours_eurostat_2.csv", encoding='utf-8')
#print(df_GDP.head())


# Choix des colonnes rélévantes
df_working_hours_selected_eurostat_2= df_working_hours_eurostat_2[['geo', 'TIME_PERIOD', 'OBS_VALUE']]
#print(df_GDP_selected.head())

# Obtention des données de panel
df_working_hours_pivot_eurostat_2 = df_working_hours_selected_eurostat_2.pivot_table(index='geo', columns='TIME_PERIOD', values='OBS_VALUE', aggfunc='first')

# Renomer l'index des colonnes
df_working_hours_pivot_eurostat_2= df_working_hours_pivot_eurostat_2.rename_axis(index={'geo': 'COUNTRY'})

# Vérification
#print(df_GDP_pivot.head())
df_working_hours_pivot_eurostat_2


In [ ]:
#1ère Juxtaposition (des deux jeux de données trimestrielles)
df_working_hours_pivot_eurostat=pd.concat([df_working_hours_pivot_eurostat_2,df_working_hours_pivot_eurostat_1], axis=1)
df_working_hours_pivot_eurostat

In [ ]:
#J'essaye de merger les données de working hours d'une autre façon, car je veux les données en format long

# suppression des données de 2008 dans un des deux df car elles sont dans des deux
df_working_hours_selected_eurostat_2 = df_working_hours_selected_eurostat_2[~df_working_hours_selected_eurostat_2['TIME_PERIOD'].isin(['2008-Q1', '2008-Q2', '2008-Q3', '2008-Q4'])]

#merger les deux l'un en dessous de l'autre
df_working_hours_long_eurostat = pd.concat([df_working_hours_selected_eurostat_2, df_working_hours_selected_eurostat_1], ignore_index="TRUE")
df_working_hours_long_eurostat


In [ ]:
#Tentative: union des deux jeux de données

#Vérifions que les pays des deux jeux de données (Eurostat + OCDE) sont identiques
identique= (
    df_working_hours_pivot_eurostat['']
    ).all()

'''idées potentielles pour la suite:
- pour corriger les missing values: peut-être on pourra faire des interpolations linéaires?
- et pour les working hours: j'ai voulu concaténer les donnés, mais mon problème c'est que je ne 
trouve pas comment ont été calculées les heures de travail par l'OCDE. Et donc j'ai du mal à faire la conversion correspondante 
J'avais pensé à utiliser les données annuelles de Eurostat
    - https://ec.europa.eu/eurostat/web/lfs/database
    - https://ec.europa.eu/eurostat/databrowser/view/lfsa_ewhuna/default/table?lang=en&category=labour.employ.lfsa.lfsa_wrktime
    Il y a des données annuelles disponibles à partir de 1998, donc c'est plus facile de faire la conversion 
    et pour 1995-1998, si on ne trouve rien, on pourra garder les données de l'OCDE (c'est plus facile de faire la conversion pour 3 ans que pour 13 ans)
    
'''

Données des taux d'intérêts

Taux court terme : "Les données se réfèrent aux bons acceptés par la banque/Certificats de dépôt négociables-3 mois."
Taux long terme : "Les données se réfèrent au taux de rendement estimé en clôture des obligations du trésor à 10 ans."

In [20]:
# Pour le long terme (une possibilité): Long term interest rates
# source: https://data-explorer.oecd.org/vis?lc=en&pg=0&tm=LONG-TERM%20INTEREST&snb=21&vw=tb&df[ds]=dsDisseminateFinalDMZ&df[id]=DSD_STES%40DF_FINMARK&df[ag]=OECD.SDD.STES&df[vs]=4.0&dq=GBR%2BROU%2BHRV%2BBGR%2BESP%2BTUR%2BCHE%2BSWE%2BSVN%2BSVK%2BPRT%2BPOL%2BNOR%2BNLD%2BLUX%2BLTU%2BLVA%2BITA%2BIRL%2BISL%2BHUN%2BGRC%2BDEU%2BFRA%2BFIN%2BEST%2BDNK%2BCZE%2BBEL%2BAUT%2BAUS.Q.IRLT.PA.....&to[TIME_PERIOD]=false&pd=1995-Q1%2C
#Unit of measure: Percent per annum

df_LT_IR=pd.read_csv('Données_extraites/Long_term_IR_OCDE.csv', encoding='utf-8')


# Choix des colonnes rélévantes
df_LT_IR_selected = df_LT_IR[['Reference area', 'TIME_PERIOD', 'OBS_VALUE']]
df_LT_IR = df_LT_IR_selected
df_LT_IR
#print(df_LT_IR.head())

# Obtention des données de panel
df_LT_IR_pivot = df_LT_IR_selected.pivot_table(index='Reference area', columns='TIME_PERIOD', values='OBS_VALUE', aggfunc='first')

# Vérification
#print(df_LT_IR_pivot.head())
#df_LT_IR_pivot


In [ ]:
# Pour le court terme (une possibilité): short term interest rates
# source: https://data-explorer.oecd.org/vis?lc=en&pg=0&tm=LONG-TERM%20INTEREST&snb=21&vw=tb&df[ds]=dsDisseminateFinalDMZ&df[id]=DSD_STES%40DF_FINMARK&df[ag]=OECD.SDD.STES&df[vs]=4.0&dq=GBR%2BROU%2BHRV%2BBGR%2BESP%2BTUR%2BCHE%2BSWE%2BSVN%2BSVK%2BPRT%2BPOL%2BNOR%2BNLD%2BLUX%2BLTU%2BLVA%2BITA%2BIRL%2BISL%2BHUN%2BGRC%2BDEU%2BFRA%2BFIN%2BEST%2BDNK%2BCZE%2BBEL%2BAUT%2BAUS.Q.IRLT.PA.....&to[TIME_PERIOD]=false&pd=1995-Q1%2C
#Unit of measure: Percent per annum

df_ST_IR=pd.read_csv('Données_extraites/Short_term_IR_OCDE.csv', encoding='utf-8')


# Choix des colonnes rélévantes
df_ST_IR_selected = df_ST_IR[['Reference area', 'TIME_PERIOD', 'OBS_VALUE']]
print(df_ST_IR.head())
df_ST_IR = df_ST_IR_selected
df_ST_IR
# Obtention des données de panel
#df_ST_IR_pivot = df_ST_IR_selected.pivot_table(index='Reference area', columns='TIME_PERIOD', values='OBS_VALUE', aggfunc='first')

# Vérification
#print(df_ST_IR_pivot.head())
#df_ST_IR_pivot


Import du CPI

In [ ]:
# données téléchargées depuis le site de eurostat

# décompresser le fichier qui était en .gz
df_cpi = pd.read_csv("estat_prc_hicp_midx_filtered_en.csv.gz", compression="gzip")
print(df_cpi.head())

# Sélectionner les variables utiles
df_cpi = df_cpi[['coicop','geo', 'TIME_PERIOD', 'OBS_VALUE']]
print(df_cpi.head())
print(df_cpi['coicop'].unique())

# mettre les deux variables de cpi différentes dans deux colonnes (car actuellement elles sont en ligne : voir la variable coicop))
df_cpi_pivot = df_cpi.pivot(index=['geo', 'TIME_PERIOD'], columns='coicop', values='OBS_VALUE')
print(df_cpi_pivot.head())
df_cpi_pivot = df_cpi_pivot.reset_index()
df_cpi_pivot


In [ ]:
#tables en format long à merger : 
set1 = set(df_working_hours_long_eurostat['geo'].unique())
set2 = set(df_cpi_pivot['geo'].unique())
set3 = set(df_ST_IR['Reference area'].unique())
set4 = set(df_LT_IR['Reference area'].unique())
set5 = set(df_GDP_selected['geo'].unique())

pays_communs = set1 & set2 & set3 & set4 & set5  # Intersection de tous les ensembles
print("Pays communs :", pays_communs)
print("Nombre de pays communs :", len(pays_communs))



In [ ]:
tous_pays = set1 | set2 | set3 | set4 | set5  # Union de tous les ensembles
print("Tous les pays :", tous_pays)

for i, (name, dataset) in enumerate([
    ("df_working_hours_long_eurostat", set1),
    ("df_cpi_pivot", set2),
    ("df_ST_IR", set3),
    ("df_LT_IR", set4),
    ("df_GDP_selected", set5)
]):
    pays_absents = tous_pays - dataset  # Pays qui ne sont pas dans cet ensemble
    print(f"Pays absents dans {name} :", pays_absents)

#suppression des US dans df_cpi_pivot :
df_cpi_pivot = df_cpi_pivot[~df_cpi_pivot['geo'].isin(['United States'])]

In [ ]:
# problème: les données sont mensuelles et elles n'existent pas en trimestriel sur eurostat : on va faire utiliser la moyenne pour chaque trimestre

# Convertir 'TIME_PERIOD' en datetime
df_cpi=df_cpi_pivot
df_cpi['TIME_PERIOD'] = pd.to_datetime(df_cpi['TIME_PERIOD'], format='%Y-%m')

# Ajouter une colonne 'quTIME_PERIODarter' en fonction de l'année et du mois
df_cpi['TIME_PERIOD'] = df_cpi['TIME_PERIOD'].dt.to_period('Q')

# Regrouper par pays et trimestre, puis prendre la moyenne
df_cpi_trimestriel = df_cpi.groupby(['geo', 'TIME_PERIOD'])[['All-items HICP','Overall index excluding energy, food, alcohol and tobacco']].mean().reset_index()
df_cpi_trimestriel['TIME_PERIOD'] = df_cpi_trimestriel['TIME_PERIOD'].astype(str).str.replace('Q', '-Q')

df_cpi_trimestriel


In [126]:
# renommer les variables de tous les df pour qu'elles soient uniformes 
df_working_hours_long_eurostat = df_working_hours_long_eurostat.rename(columns={'geo': 'country', 'OBS_VALUE': 'working_hours'})
df_cpi_trimestriel = df_cpi_trimestriel.rename(columns={'geo': 'country'})
df_ST_IR = df_ST_IR.rename(columns={'Reference area': 'country', 'OBS_VALUE': 'short term'})
df_LT_IR = df_LT_IR.rename(columns={'Reference area': 'country', 'OBS_VALUE': 'long term'})
df_GDP_selected = df_GDP_selected.rename(columns={'geo': 'country', 'OBS_VALUE': 'gdp'})




In [ ]:
# Maintenant on merge

from functools import reduce


# Liste de DataFrames à merger
dfs = [df_GDP_selected, df_ST_IR, df_LT_IR, df_working_hours_long_eurostat,df_cpi_trimestriel ]  

# Fusionner tous les DataFrames de la liste
df_merged = reduce(lambda left, right: left.merge(right, on=['country', 'TIME_PERIOD'], how='outer'), dfs)
df_merged

